# Graph Spectral Clustering

In this notebook, we will explore the following questions: What is the impact each individual eigenvector has on the normalized spectral clustering algorithm? While incrementing $k$, the number of clusters, how do the clusterings change? What kinds of datasets can we cluster well using spectral clustering? There will be some sections for you to fill in missing code, as well as some short answers based on the output of your code. (For all plotting questions, please include a title and axis labels.)

In [ ]:
import matplotlib.pyplot as plt
import warnings
import matplotlib.cbook

warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

import numpy as np
import scipy as sp
from scipy import linalg
from matplotlib import cm
from tqdm import tqdm
from sklearn.cluster import KMeans, SpectralClustering

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, fixed, HBox, Layout, VBox, interactive, Label

## Part 1: Data generation

First, let's generate the first set of data we will use for clustering. This data will be drawn from a gaussian mixture.

For k clusters, the gaussian mixture is generated by first choosing `n_gaussian` centroids, randomly according to a normal distribution with variance `centroid_var`. For each of these centroids, the `n_pts` points are then chosen according the normal distribution with variance `cluster_var`. 

**Question 1a.** Implement the lines below in the `gaussian_mixture` function to generate data points centered around `n_gaussians` number of centroids, as described above.

In [ ]:
def gaussian_mixture(n_gaussians, n_pts, dim, centroid_var=5, cluster_var=1):
    '''
    Return a matrix of gaussian vectors, one per row, where each column is a feature.
    n_gaussians: integer - number of gaussians to use in the GMM
    n_pts: integer - number of points to sample per gaussian
    dim: integer - dimensionality of points
    centroid_var: float - variance used to pick random means of each Gaussian
    cluster_var: float - variance used for each Gaussian of the mixture
    '''

    # Generate centroids of size (n_gaussians x dim), drawn from a normal
    # distribution with mean 0 and variance centroid_var.
    ### BEGIN YOUR SOLUTION ###

    
    
    ### END YOUR SOLUTION ###

    points_by_gaussian = [
    # For each gaussian, sample the points and add the centroid mean.
    ### BEGIN YOUR SOLUTION ###

        
        
    ### END YOUR SOLUTION ###
        for i in range(n_gaussians)
    ]

    # Join the points as rows.

    return np.vstack(points_by_gaussian)

Carefully read over and understand the cell below. It will generate sliders to visualize the data we just generated above. Learning how to properly plot and visualize our data is important, as it can provide a quick way to check our work. Then, run the cell below to visualize the data we just generated. Play around with the sliders to see how changing the parameters affect the data that is being generated. As a sanity check, the data generated should match the description provided above.

In [ ]:
def interactive_gaussian(n_gaussians=5, n_pts=4, d=2, centroid_var=5, cluster_var=1):
    n = n_pts * n_gaussians
    data = gaussian_mixture(n_gaussians, n_pts, d, centroid_var)
    plt.scatter(*data.T)
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.title("Sample of Gaussian Mixture")
    plt.show()

widget=interactive(interactive_gaussian, {'manual': True},
                  n_gaussians=(2, 6, 1),
                  n_pts=(2, 10, 1),
                  d=fixed(2),
                  centroid_var=(1,10,1),
                  cluster_var=(1,20,1))

widget.children[-2].description='Show Plot'
widget.children[-2].style.button_color='lightblue'
controls = HBox(widget.children[:-1], layout=Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))


**Question 1b.** Describe how each of the values that are being changed by the sliders affect the plot generated. Does this match up with what you expect? If not, revisit your code above to make sure the data is being generated properly.

**Answer:**

_Insert answer here_

**Question 1c.** Finally, let's generate the data that we will work with for parts 2 and 3 below. Use the following parameters for the data:

<center>n_gaussians=5, n_pts=4, dim=2, centroid_var=10, cluster_var=1</center><br>

Then, plot the data. Make sure the data generated matches what you would expect.

In [ ]:
### BEGIN YOUR SOLUTION ###
n_gaussians = _
n_pts = _
n = _  # the total number of data points (used later)
d = _
centroid_var = _
cluster_var = _

data = _

plt.scatter(*data.T)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.title("Sample of Gaussian Mixture")
plt.show()
### END YOUR SOLUTION ###

## Part 2: The Spectral Clustering Algorithm

We will now get into the heart of the spectral clustering algorithm. Below, we have some methods that will help you further understand how the algorithm works. Some of the methods will require some code to be filled in.

This section will walk you through the steps of take your data points to create a normalized symmetric Laplacian matrix, stack the eigenvectors corresponding to the smallest eigenvalues into a matrix, then cluster the rows of this new matrix using k-means. From the cluster assignments found by k-means, we can then recover the clusters for our original points.

**Note:** This section should take the majority of time. Referring back to notes/lecture material will help you understand how the algorithms below work, and what to implement. It will save you time to first understand the algorithms before jumping into implementation.

The k-means algorithm is as follows:
1. Initialize $k$ points $c_i\in \mathbb{R}^d$ with $i=1,\dots,k$ to be the centroids of each cluster. Initializing with k-means++ picks these centroids to be as distant as possible from each other, to help ensure the correct number of clusters. More details on this in the next section.
2. Assign each data point to the closest centroid. The points corresponding to the same centroid form the clusters.
3. Move each centroid to the center of its current cluster (using the updated assignments of data).
4. Iterate steps 2 and 3 until updating the centroids in step 3 no longer changes the cluster assignments in step 2.

We improve upon this k-means algorithm by changing the initialization process. In regular k-means, the centroids are initialized randomly. It has been shown that under random initialization, the worst case runtime can be superpolynomial in runtime (not bounded above by any polynomial, bad) and the approximation found can be arbitrarily bad in comparison to the optimal clustering.

So how does the k-means++ initialization of centroids work? The first centroid is chosen arbitrarily at random and every subsequent point is chosen from the remaining points with probability proportional to the square distance from the closest already initialized centroid. It turns out that this initialization guarantees a O(log k) approximation in expectation where k is the number of clusters.

**Question 2a.** Fill in the missing code below in the `kmeans` function according to the pseudocode provided above.

In [ ]:
def kmeans(data, k, iters=100, init="++"):
    '''
    data: np.ndarray - a [n, d] numpy array of n unique d-dimensional datapoints
    k: integer - number of clusters to compute
    init: string - one of ["random", "++"] to either initialize with centroids drawn from a random Gaussian, or to use kmeans++
    '''
    n, d = data.shape

    def new_assns(centroids, as_dist=False):
        ### BEGIN YOUR SOLUTION ###
        # List of [norm(data_vector[i], centroid[j]) for j=1..k]

        
        
        
        ### END YOUR SOLUTION ###

        ### BEGIN YOUR SOLUTION ###
        # find index of the nearest centroid for each point
        # (argmin returns the *index* of the minimum item, rather than the item itself)

        
        
        
        
        ### END YOUR SOLUTION ###

    def new_centroids(assns, centroids):
        ### BEGIN YOUR SOLUTION ###
        # for j=1..k, find all data columns matching this assignment, then average the cols

        
        
        
        
        
        
        ### END YOUR SOLUTION ###
        return np.vstack(candidate_centroids)

    assns = np.zeros(n)

    if(init == "random"):
        ### BEGIN YOUR SOLUTION ###
        # Initialize k x d centroids drawn from a normal distribution with the same variance as the sample variance.

        
        
        
        ### END YOUR SOLUTION ###
    elif(init == "++"):
        ### BEGIN YOUR SOLUTION ###
        # Initialize k x d centroids through kmeans++

        
        
        
        
        
        ### END YOUR SOLUTION ###
    else:
        raise ValueError("Initialization must be one of ['random', '++']")


    for _ in range(iters):
        assns = new_assns(centroids)
        centroids = new_centroids(assns, centroids)
    return centroids, assns

**Question 2b.** In a real world scenario, you likely won't be required to implement k-means by hand, but instead use sklearn's built-in k-means function. To make sure you understand how to use that function, write one line of code below for how you would hypothetically cluster using sklearn's k-means function (feel free to look up documentation online).

In [ ]:
### BEGIN YOUR SOLUTION ###

### END YOUR SOLUTION ###

In the next parts, you will implement functions needed for spectral graph clustering. As a reminder, here is the pseudocode for the algorithm:
1. Construct a similarity graph according to some well-defined metric (e.g. the Gaussian kernel, Euclidean distance, etc.).
2. Compute $L = D - A$, the unnormalized graph Laplacian.
3. Compute the symmetric normalized graph Laplacian $L_{sym} = D^{-1/2}LD^{-1/2}$
4. Compute the first $k$ eigenvectors of $L_{sym}$, where the first $k$ eigenvectors correspond to the $k$ smallest eigenvalues.
5. Construct a matrix $H$ for $L_{sym}$ with the k eigenvectors as the columns, making sure to normalize the rows to have norm 1.
6. Run k-means on the rows of $H$ to cluster the vertices.

**Question 2c.** Fill in the code below. Set the kernel to either the gaussian or exponential kernel, then populate entries in the similarity matrix (remember that diagonals should be zero).

In [ ]:
def similarity_matrix(data, s=1, metric="g", kernel=None, numOfAtts=None):
    '''
    Compute an adjacency matrix given a data matrix of size (n, d). Similarities are computed using a kernel specified by the `metric` argument. 
    data: np.ndarray - (n, d) numpy array containing n datapoints having d dimensions
    s: float - scale parameter to be used in each metric. The effect of `s` depends on choice of metric.
    metric: one of ["g", "e", "k"] - choose a metric for the data:
        "g" for Gaussian: d(x, y) = exp(-|x-y|^2 / 2 (s^2)). The scale `s` controls standard deviation. 
        "e" for Exponential: d(x, y) = exp(-|x-y|/s). The scale `s` is the parameter of the exponential.
        "eps" for Epsilon neighbors: d(x, y) = |x-y| if less than epsilon, 0 otherwise
        "k" for Kernel: use an arbitrary kernel, given by the `kernel` argument.
    kernel: K(x, y, s) -> \R+ - an arbitrary distance function between two points of the same dimension with a given scale.

    @returns an (n, n) np.ndarray adjacency matrix representing a similarity graph for the data
    '''
    n = len(data)
    similarity_matrix = np.zeros((n, n))

    if(metric == "g"):
        # assign `kernel` to the gaussian kernel
        ### BEGIN YOUR SOLUTION ###

        
        
        ### END YOUR SOLUTION ###
    elif(metric == "e"):
        # assign `kernel` to the exponential kernel
        ### BEGIN YOUR SOLUTION ###

        
        
        ### END YOUR SOLUTION ###
    elif(metric == "eps"):
        def eps_threshold(x, eps):
            return x if x < eps else 0
        kernel = lambda x, y, s : eps_threshold(np.linalg.norm(x-y), s)
    elif(metric == "k"):
        assert kernel is not None, "Must pass a kernel function to use kernelized similarity metric"
    else:
        raise ValueError("Similarity metric must be one of [g, e, k]")


    for i in range(n):
        for j in range(n):
            ### BEGIN YOUR SOLUTION ###

            
            
            
            ### END YOUR SOLUTION ###

    return similarity_matrix

**Question 2d.**  Complete the function to return a Laplacian matrix given the adjacency matrix of a graph.

In [ ]:
def laplacian_matrix(graph_weights):
    '''
    Return a graph laplacian and degree matrix given the graph's edge weights
    graph_weights: np.ndarray - an (n, n) dense graph adjacency matrix
    @returns a tuple (laplacian, degree) containing the (n, n) graph laplacian and the (n,) diagonal elements of the degree matrix
    '''
    ### BEGIN YOUR SOLUTION ###

    
    
    ### END YOUR SOLUTION ###
    return laplacian, degree

**Question 2e.** Compute the eigenvalues and normalized eigenvectors for the symmetric Laplacian.

In [ ]:
def spectral_clustering(data, k, with_eigen = False, kmeans_iters = 100, numOfAtts=None, metric = None, **kwargs):
    '''
    Args:
        data (np.ndarray): (n,d) numpy array consisting of n d-valued points
        k (integer): desired number of clusters
        with_eigen (:obj:bool, optional) - if True, will also return a tuple (evalues, evecs) of the k Laplacian eigenpairs
        metric: one of ["g", "e", "k"] - choose a metric for the data:
            "g" for Gaussian: d(x, y) = exp(-|x-y|^2 / 2 (s^2)). The scale `s` controls standard deviation.
            "e" for Exponential: d(x, y) = exp(-|x-y|/s). The scale `s` is the parameter of the exponential.
            "k" for Kernel: use an arbitrary kernel, given by the `kernel` argument.
    Returns:
        A list of (n,) integers of the cluster assignments of each data point. If with_eigen=True, also returns eigenvalues and eigenvectors of the Laplacian.
    1. use a specialized algorithm to compute indicator vectors
    2. cluster the eigenvectors with k-means
    ''' 

    data_sim = similarity_matrix(data, **kwargs, numOfAtts = numOfAtts)
    n,d = data_sim.shape
    laplacian, degree = laplacian_matrix(data_sim)

    ### BEGIN YOUR SOLUTION ###
    # compute the symmetric laplacian
                                                # compute D^(-1/2)
                                                # Construct the symmetric laplacian
    # compute the first k eigenvectors/eigenvalues of lsym
    # note that because it is symmetric, we can do this efficiently with sp.linalg.eigh
                                                # S, U = sp.linalg.eigh(__, eigvals=__)
                                                # normalize rows of the matrix
    ### END YOUR SOLUTION ###

    centroids, assns = kmeans(U, k, iters=kmeans_iters)
    if(with_eigen):
        return (assns, (S, U))
    else:
        return assns

**Question 2f.** Again, implement spectral clustering below, using sklearn. Print the labels of the data generated from the Gaussian mixture.

In [ ]:
### BEGIN YOUR SOLUTION ###



### END YOUR SOLUTION ###

## Part 3: Extracting and Visualizing Eigenvectors

Now, let's visualize the eigenvectors. The following plot shows eigenvectors (of the Graph Laplacian) in increasing order of the corresponding eigenvalues. The spectral clustering function is called with the exponential kernel (one of the similarity kernels that can be used for spectral clustering), to provide a graph visualizing the eigenvectors.

The color map used is viridis. Purple represents large negative numbers, and it ranges up in backwards rainbow order to yellow, which represents large positive numbers.

For the following questions, comment your observations about the extraction and visualization of the eigenvectors that just occurred. 

In [ ]:
assns, _ = spectral_clustering(data, k=5, with_eigen=True, metric="e")
print("The cluster assignments for 5 clusters: ", assns)
_, (evals, evecs) = spectral_clustering(data, k=n, with_eigen=True, metric="e")
plt.imshow(evecs)
# print(evecs) ## uncomment this part of the code to see what the entries of the matrix are
plt.axis('off')
plt.title("Eigenvectors of Graph Laplacian")
plt.show()

**Question 3a.** Based on this visualization, what do you notice about the eigenvectors of the graph laplacian?

**Answer:**

_Insert answer here_


**Question 3b.** Next, plot the eigenvalues of the similarity graph Laplacian. You should generate a plot showing eigenvalue growth rate. 

In [ ]:
### BEGIN YOUR SOLUTION ###






### END YOUR SOLUTION ###

**Question 3b.** What do you notice about the eigenvalues? (Hint: how many eigenvalues are nearly zero?) How can looking at the eigenvalue plot inform us about good $k$'s to choose?

**Answer:**

_Insert answer here_

### Cumulative Clusters

The cells below will generate subplots, which show classification decisions using a certain number of eigenvectors of the normalized graph Laplacian. These are called cumulative clusters, in the sense that plot $(i+1)$ includes all vectors used to cluster plot $(i)$, plus an additional vector.

In [ ]:
cmap = cm.get_cmap("tab20")
# Note: if n > 20, then multiple indices end up in the same color bin, inducing a seemingly bad clustering
unif_colors = [cmap(intensity) for intensity in np.linspace(0, 1, n)] 

In [ ]:
r = n_pts
c = n_gaussians
# r * c = N
for i in tqdm(range(1,n+1)):
    #evecs = np.fliplr(evecs)
    _, assns = kmeans(evecs[:, :i], i, iters=100)
    data_clusters = [ data[assns == clss].T for clss in range(i) ]
    plt.subplot(r, c, i)
    plt.title(f"{i} evecs")
    plt.gca().set_xticks([], [])
    plt.gca().set_yticks([], [])

    for j, data_cluster in enumerate(data_clusters):
        plt.scatter(*data_cluster, color=unif_colors[j])

plt.gcf().set_size_inches(10, 10)
plt.tight_layout()
plt.show()

**Question 3c.** How is the number of clusters $k$ related to how well the algorithm clusters? Which $k$ does the best in this case and why?

**Answer:**

_Insert answer here_

It's important to understand how these cluster assignments come to be from the eigenvectors of the graph laplacian. We cluster the rows of this matrix using k means to get the cluster assignments.

It's also important to note that we used k-means++ to cluster the rows of our eigenvector matrix instead of the canonical k-means. The difference lies in the convergence times. In k-means++ we initialize the centroids to be as far away as possible and this simple change in initialization helps us reach good solutions. Sometimes if we get really unlucky with our initialization for k-means we end up with either the wrong cluster assignments, or it takes a lot longer to converge to the correct solution.

## Part 4: Classifying Harder Datasets

For the last part, we will move on to classifying more difficult datasets, in which the clusters are not as obvious. We will observe how well k-means and spectral clustering can identify the moons dataset. From the sklearn documentation, this is a 'simple toy dataset to visualize clustering and classification algorithms. Enjoy!

In [ ]:
from sklearn.datasets import make_moons

**Question 4a.** Plot a scatterplot of the moons data using the `make_moons` function from sklearn.

In [ ]:
moon_data, _ = make_moons(200, noise=0.05)

### BEGIN YOUR SOLUTION ###





### END YOUR SOLUTION ###

**Question 4b.** Based on your visualization, why do you think clustering techniques like k-means will not work on the moons dataset? Why is spectral graph clustering suitable to use on this dataset?

**Answer:**

_Insert answer here_

Run the next cell to see how K-means will cluster the two moons dataset. Make sure your answer, and the plot below line up.

In [ ]:
_, assns = kmeans(moon_data, k=2) # _, assns = 

moon_data_clusters = [ moon_data[assns == clss].T for clss in range(n_gaussians) ]
for j, moon_data_cluster in enumerate(moon_data_clusters):
    plt.scatter(*moon_data_cluster, color=unif_colors[j], label=f"{j}")

plt.title("Clustering with K-means")

**Question 4c.** Fill in the code below to implement a slider for values of $\sigma$, which range from 0.01 to 1, with step size of 0.01. Play around with the slider. You should be able to find a range of values which will cluster the two moons dataset correctly.

In [ ]:
def interactive_gaussian_kernel(data=moon_data, sigma=0.01):
    ### BEGIN YOUR SOLUTION ###

    
    
    
    
    ### END YOUR SOLUTION ###
    plt.title("Spectral Clustering with Gaussian Kernel, $\sigma=$" + str(sigma))

### BEGIN YOUR SOLUTION ###








### END YOUR SOLUTION ###

**Question 4d.** Comment your observations on how the hyperparameter $\sigma$ affects the cluster assignments. What values of $\sigma$ cluster the two moons correctly?

**Answer:**

_Insert answer here_

**Question 4e.** How do the values of $\sigma$ affect the cluster assignments?

**Answer:**

_Insert answer here_